IMPORTS

In [29]:
from google.colab import drive
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import math

LOADING DATA

In [6]:
drive.mount('/content/gdrive')
%cd gdrive/My Drive/Colab Notebooks/IC23/

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/IC23


In [52]:
#! git clone https://github.com/osullik/ic23050.git
#! git pull

In [3]:
orig_data = pd.read_csv("https://raw.githubusercontent.com/osullik/ic23050/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/Data_Washington%20Fatal%20Crash%20Survey.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (21,22,23,301,303) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
orig_data.head()

,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,holiday,county,...,nmdistract6,lab,isedtcase,crf1,crf2,crf3,CoRoadName,CoMP,IntCoRoadName,IntCoMP
0,2017,1,E628946,2.0,01/01/2017,2:12,1,1,1.0,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,2,E627989,26.0,01/02/2017,17:14,2,1,1.0,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,4,3747633,263.0,01/01/2017,18:47,1,1,1.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,5,E628691,4.0,01/01/2017,3:50,1,1,1.0,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,6,3746306,263.0,01/05/2017,9:53,5,1,0.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


REVERSE GEOCODING

In [11]:
locator = Nominatim(user_agent="myGeocoder")

In [20]:
# Input: lat and long coordinates
# Output: dictionary containing 'address' -> 'postcode' and other address fields
def reverse_geo(x, y):
  try:
    return locator.reverse((x,y)).raw
  except ValueError:
    print(x, y)

In [41]:
# Input: geo dictionary
# Output: zipcode
def get_zipcode(geodict):
  try:
    return geodict['address']['postcode']
  except KeyError:
    return np.nan

In [13]:
#x, y = 40.7128, -74.0060
#reverse_geo(x, y)

{'place_id': 226179173,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 575213527,
 'lat': '40.71273945',
 'lon': '-74.00593904130275',
 'display_name': 'New York City Hall, 260, Broadway, Lower Manhattan, Manhattan Community Board 1, Manhattan, New York County, City of New York, New York, 10000, United States',
 'address': {'amenity': 'New York City Hall',
  'house_number': '260',
  'road': 'Broadway',
  'quarter': 'Lower Manhattan',
  'neighbourhood': 'Manhattan Community Board 1',
  'suburb': 'Manhattan',
  'county': 'New York County',
  'city': 'City of New York',
  'state': 'New York',
  'ISO3166-2-lvl4': 'US-NY',
  'postcode': '10000',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['40.712445', '40.7130254', '-74.0064455', '-74.0055687']}

In [43]:
orig_data['zipcode'] = orig_data.apply(lambda row: get_zipcode(reverse_geo(row['y'], row['x'])), axis=1)

In [51]:
orig_data.to_csv('/content/gdrive/My Drive/Colab Notebooks/IC23/Data_Washington_Fatal_Crash_Survey_with_zip.csv')

AUGMENTING WITH OSM DATA

In [1]:
# !pip install pyrosm

In [2]:
# from pyrosm import OSM
# from pyrosm import get_data
# from pyrosm.data import sources

In [3]:
# data_with_zip = pd.read_csv('https://raw.githubusercontent.com/osullik/ic23050/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/Data_Washington_Fatal_Crash_Survey_with_zip.csv')

In [4]:
# # Grab OSM data from pyrosm
# fp = get_data("washington") # AKA sources.north_america.usa.washington also at '/content/gdrive/My Drive/Colab Notebooks/IC23/washington-latest.osm.bz2'

# # Initialize the OSM parser object
# osm = OSM(fp)

In [ ]:
# # Read all drivable roads
# drive_net = osm.get_network(network_type="driving")
# drive_net.plot()

In [ ]:
# drive_net.head(2)

AUGMENT ZIPCODES WITH DEMOGRAPHICS

In [10]:
# !pip install uszipcode

In [190]:
from uszipcode import SearchEngine  # data.census.gov

In [239]:
# Augment with extra demographic fields based on zipcode
# skip any that don;t actually show up in the SearchEngine results
# skip county because it's already in the crash data
fields_to_add = ('state', 'population', 'population_density', 'land_area_in_sqmi', 'water_area_in_sqmi', 'housing_units', 'occupied_housing_units', 'median_home_value', 'median_household_income')
fields_to_add_list = list(fields_to_add)

def augment_zip(zipcode):
  try:
    s = SearchEngine().by_zipcode(zipcode)
    return [s.state, s.population, s.population_density, s.land_area_in_sqmi, s.water_area_in_sqmi, s.housing_units, s.occupied_housing_units, s.median_home_value, s.median_household_income]  # fields listed here: https://uszipcode.readthedocs.io/
  except Exception:
    return [np.nan for _ in range(len(fields_to_add))]  

In [246]:
# Read in augmented data with zipcodes
orig_data_with_zip = pd.read_csv("https://raw.githubusercontent.com/osullik/ic23050/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/Data_Washington_Fatal_Crash_Survey_with_zip.csv")

# Drop all rows with mission driver zipcode or zipcode
orig_data_with_zip = orig_data_with_zip[orig_data_with_zip['zipcode'].notna()] 
orig_data_with_zip = orig_data_with_zip[orig_data_with_zip['dzip'].notna()]  
orig_data_with_zip = orig_data_with_zip.astype({'dzip':'int', 'zipcode':'int'})

In [247]:
orig_data_with_zip[fields_to_add] = orig_data_with_zip.apply(lambda row: augment_zip(row['zipcode']), axis=1)
orig_data_with_zip[fields_to_add_list] = pd.DataFrame(orig_data_with_zip[fields_to_add].tolist(),index=orig_data_with_zip.index)
orig_data_with_zip = orig_data_with_zip.drop(fields_to_add, axis=1)

Exception ignored in: <function SearchEngine.__del__ at 0x7ff631042ca0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/uszipcode/search.py", line 196, in __del__
    self.close()
  File "/usr/local/lib/python3.8/dist-packages/uszipcode/search.py", line 202, in close
    self.ses.close()
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 1811, in close
    self._close_impl(invalidate=False)
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 1853, in _close_impl
    transaction.close(invalidate)
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 923, in close
    transaction.close()
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2416, in close
    self._do_close()
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2639, in _do_close
    self._close_impl()
  File "/usr/local/lib/python3.8/dist-packages/sqlal

DATA CLEANING

In [248]:
# Drop any columns or rows with more than 50% Nans
orig_data_with_zip.dropna(thresh=len(orig_data_with_zip.index)*0.5, axis=1, inplace=True)
orig_data_with_zip.dropna(thresh=len(orig_data_with_zip.columns)*0.5, axis=0, inplace=True)

# Drop any rows with invalid zipcode (not 5 digits)
tmp = orig_data_with_zip.astype({'zipcode':'str'})
mask = (tmp['zipcode'].str.len() == 5)
orig_data_with_zip = orig_data_with_zip.loc[mask]

# More cleaning here ...

orig_data_with_zip

,Unnamed: 0,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,county,...,zipcode,state,population,population_density,land_area_in_sqmi,water_area_in_sqmi,housing_units,occupied_housing_units,median_home_value,median_household_income
0,0,2017,1,E628946,2.0,01/01/2017,2:12,1,1,61,...,98201,WA,29582.0,3956.0,7.48,2.68,13899.0,12443.0,236800.0,43229.0
1,1,2017,2,E627989,26.0,01/02/2017,17:14,2,1,35,...,98359,WA,4887.0,349.0,14.02,1.25,1982.0,1821.0,275500.0,71676.0
2,2,2017,4,3747633,263.0,01/01/2017,18:47,1,1,33,...,98103,WA,45911.0,9905.0,4.64,0.66,24128.0,22724.0,478200.0,75763.0
3,3,2017,5,E628691,4.0,01/01/2017,3:50,1,1,63,...,99219,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2017,6,3746306,263.0,01/05/2017,9:53,5,1,33,...,98174,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4126,4126,2021,636,Incid08,334.0,08/08/2021,9:38,8,8,47,...,99155,WA,1350.0,4.0,304.50,3.25,443.0,402.0,79100.0,25435.0
4127,4127,2021,636,Incid08,334.0,08/08/2021,9:38,8,8,47,...,99155,WA,1350.0,4.0,304.50,3.25,443.0,402.0,79100.0,25435.0
4129,4129,2021,639,EB48605,263.0,04/21/2021,17:32,21,4,33,...,98144,WA,26881.0,7895.0,3.40,1.75,12660.0,11511.0,391000.0,55950.0
4130,4130,2021,639,EB48605,263.0,04/21/2021,17:32,21,4,33,...,98144,WA,26881.0,7895.0,3.40,1.75,12660.0,11511.0,391000.0,55950.0


In [249]:
for i in orig_data_with_zip.columns:
  print(i)

Unnamed: 0
year
case
par
repjur
crash_dt
crash_tm
accday
accmon
county
co_char
city
weather
weather2
lightcond
surfcond
surftype
numfatal
nmforms
pforms
vforms
fhevent
hitrun
manncol
milepnt
reljunc
reljuncinter
intersectiontype
funcsystem
landuse
ownership
route
schlbus
spejur
contdev
devfunc
trafid1
x
y
wk_zone
vnumber
crashtype
criticalevent
criticaleventcat
spdlim
body
trailer
emerg
hazcargo
leave
mhevent
roadalgn
roadprof
traflane
trafflow
vehmanvr
numoccs
regowner
regstate
spuse
make
model
modelyr
vrf1
vrf2
seqevent1
pnumber
ptype
age
sex
hispanic
injury
diedscene
restraint
restraintmisuse
seatpos
takehosp
commlic
licendor
licrestrict
liccomp
drivpres
drf1
drf2
drf3
drf4
distract1
dricond1
dzip
licstate
drhgt_in
weight
noncdl
noncdltype
prevdwi
prevoth
prevacc
prevspd
driviol1
spdrel
first_dt
last_dt
airbag
ejection
ejectpath
extricat
injurwrk
methalc
methdrug
prf1
prf2
prf3
druginv
alcinvol
drugsts
drugres1
drugtst1
alcsts
alctst
alcres
train_inv
wildlife_inv
rotr_fars
intersect

ANALYSIS Q1 - Proportion of people involved in crashes where dzip differes from zipcode

In [261]:
q1 = orig_data_with_zip[['case','crash_dt','crash_tm','dzip','zipcode']]
q1 = q1.dropna()

q1_local = q1.loc[~(q1['dzip'] == q1['zipcode'])]
q1_foreign = q1.loc[~(q1['dzip'] != q1['zipcode'])]

print('Num local crashes:', len(q1_local['case']))
print('Num foreign crashes:', len(q1_foreign['case']))
print('Num crashes:', len(q1['case']))

Num local crashes: 2846
Num foreign crashes: 862
Num crashes: 3708


ANALYSIS Q1A - Zipcodes producing drivers most likely to be involved in a crash

In [260]:
q1a = orig_data_with_zip[['case','crash_dt','crash_tm','dzip','population']]
q1a = q1a.dropna()

# reindex so dzip is unique, measure counts of rows per dzip, carry over population based on corresponding dzip

,case,crash_dt,crash_tm,dzip,population
0,1,01/01/2017,2:12,98204,29582.0
1,2,01/02/2017,17:14,98465,4887.0
2,4,01/01/2017,18:47,98125,45911.0
8,8,01/07/2017,14:12,98311,919.0
9,8,01/07/2017,14:12,98934,919.0
...,...,...,...,...,...
4126,636,08/08/2021,9:38,98840,1350.0
4127,636,08/08/2021,9:38,99155,1350.0
4129,639,04/21/2021,17:32,98503,26881.0
4130,639,04/21/2021,17:32,98144,26881.0
